In [ ]:
#this is a template to adapt to my usecase
#old and useless

In [ ]:

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
import random
import numpy as np


In [ ]:

# ── 1. Reproducibility ───────────────────────────────────────────────────────────
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
# ── 2. Device ───────────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
# ── 3. Data Preparation ─────────────────────────────────────────────────────────
# Transforms
transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.5071, 0.4867, 0.4408),
                (0.2675, 0.2565, 0.2761)),
])
transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5071, 0.4867, 0.4408),
                (0.2675, 0.2565, 0.2761)),
])

In [ ]:
# Download & split
dataset_full = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)
val_size = 5000
train_size = len(dataset_full) - val_size
train_dataset, val_dataset = random_split(
    dataset_full, [train_size, val_size],
    generator=torch.Generator().manual_seed(seed))
test_dataset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

100%|██████████| 169M/169M [00:04<00:00, 40.6MB/s]


In [ ]:
# DataLoaders (batch_size=64 per paper)
batch_size = 64
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
# ── 4. Model Definition ─────────────────────────────────────────────────────────
class LELeNetCIFAR(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1   = nn.Linear(64 * 8 * 8, 384)
        self.fc2   = nn.Linear(384, 192)
        self.fc3   = nn.Linear(192, 100)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

model = LELeNetCIFAR().to(device)

In [ ]:
# ── 5. Optimizer, Scheduler, Loss ────────────────────────────────────────────────
optimizer = optim.SGD(
    model.parameters(),
    lr=0.01,           # per paper
    momentum=0.9,      # per paper
    weight_decay=4e-4  # per paper
)
num_epochs = 100
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
criterion = nn.CrossEntropyLoss()

In [ ]:
# ── 6. Logging & Checkpointing Setup ────────────────────────────────────────────
writer = SummaryWriter(log_dir='./logs')
ckpt_dir = './checkpoints'
os.makedirs(ckpt_dir, exist_ok=True)

def save_checkpoint(epoch, best=False):
    path = os.path.join(
        ckpt_dir,
        f'{"best" if best else "last"}_ckpt_epoch_{epoch}.pth')
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optim_state': optimizer.state_dict(),
        'sched_state': scheduler.state_dict(),
        'rng_state': torch.get_rng_state(),
    }, path)

def load_checkpoint(path):
    ckpt = torch.load(path)
    model.load_state_dict(ckpt['model_state'])
    optimizer.load_state_dict(ckpt['optim_state'])
    scheduler.load_state_dict(ckpt['sched_state'])
    torch.set_rng_state(ckpt['rng_state'])
    return ckpt['epoch']

In [ ]:
# ── 7. Training & Evaluation Functions ─────────────────────────────────────────
def train_one_epoch():
    model.train()
    running_loss = correct = total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def eval_model(loader):
    model.eval()
    running_loss = correct = total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

In [ ]:
# ── 8. Main Training Loop ───────────────────────────────────────────────────────
best_val_acc = 0.0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc = eval_model(val_loader)

    scheduler.step()
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f"Epoch {epoch}/{num_epochs}  "
          f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}  "
          f"Val:   loss={val_loss:.4f}, acc={val_acc:.4f}")

    writer.add_scalars('Loss', {'train': train_loss, 'val': val_loss}, epoch)
    writer.add_scalars('Acc',  {'train': train_acc,  'val': val_acc},   epoch)

    # Checkpoint
    save_checkpoint(epoch, best=False)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_checkpoint(epoch, best=True)

Epoch 1/100  Train: loss=4.1107, acc=0.0678  Val:   loss=3.7990, acc=0.1128
Epoch 2/100  Train: loss=3.5657, acc=0.1488  Val:   loss=3.3450, acc=0.1922
Epoch 3/100  Train: loss=3.2182, acc=0.2091  Val:   loss=3.1327, acc=0.2260
Epoch 4/100  Train: loss=2.9821, acc=0.2538  Val:   loss=2.8528, acc=0.2862
Epoch 5/100  Train: loss=2.7878, acc=0.2925  Val:   loss=2.7273, acc=0.2984
Epoch 6/100  Train: loss=2.6293, acc=0.3254  Val:   loss=2.6252, acc=0.3314
Epoch 7/100  Train: loss=2.5032, acc=0.3521  Val:   loss=2.6146, acc=0.3478
Epoch 8/100  Train: loss=2.4114, acc=0.3720  Val:   loss=2.4780, acc=0.3586
Epoch 9/100  Train: loss=2.3255, acc=0.3911  Val:   loss=2.5215, acc=0.3570
Epoch 10/100  Train: loss=2.2452, acc=0.4074  Val:   loss=2.4799, acc=0.3674
Epoch 11/100  Train: loss=2.1773, acc=0.4223  Val:   loss=2.3929, acc=0.3794
Epoch 12/100  Train: loss=2.1189, acc=0.4340  Val:   loss=2.3660, acc=0.3910
Epoch 13/100  Train: loss=2.0663, acc=0.4450  Val:   loss=2.3330, acc=0.4008
Epoch 14

In [ ]:
# ── 9. Final Test Evaluation ───────────────────────────────────────────────────
test_loss, test_acc = eval_model(test_loader)
print(f"\nTest: loss={test_loss:.4f}, acc={test_acc:.4f}")

NameError: name 'eval_model' is not defined

In [ ]:
# ── 10. Plot Accuracies ─────────────────────────────────────────────────────────
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_acc'],   label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()